# Spectral Clustering  

In this notebook, we will use cuGraph to identify the cluster in a test graph using the Spectral Clustering Balance Cut algorith,  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0

Test Hardware
* GP100 32G, CUDA 9,2


## Introduction

Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a given number of clusters which satisfy a given quality metric. Balanced Cut is such a quality metric. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the balanced cut metric in cugraph use:
__cugraph.spectralBalancedCutClustering(G, num_clusters, num_eigen_vects)__

Input
* __G__: A cugraph.Graph object
* __num_clusters__: The number of clusters to find
* __num_eig__: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.


## cuGraph 0.6 Notice 
cuGraph version 0.6 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations 


## References


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from scipy.io import mmread
from collections import OrderedDict

### Read the CSV datafile using cuDF

In [9]:
# Test file  
datafile='../data/networks/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

### Adjusting the vertex ID
Let's adjust all the vertex IDs to be zero based.  We are going to do this by adding two new columns with the adjusted IDs

In [10]:
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [11]:
# The algorithm requires that there are edge weights.  In this case all the weights are being ste to 1
gdf["data"] = cudf.Series(np.ones(len(gdf), dtype=np.float32))

In [12]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

,src,dst,src_0,dst_0,data
0,1,2,0,1,1.0
1,1,3,0,2,1.0
2,1,4,0,3,1.0
3,1,5,0,4,1.0
4,1,6,0,5,1.0


In [13]:
# verify data type
gdf.dtypes

src        int32
dst        int32
src_0      int32
dst_0      int32
data     float32
dtype: object

Everything looks good, we can now create a graph

In [14]:
# create a CuGraph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [15]:
# Call spectralBalancedCutClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralBalancedCutClustering(G, 3, num_eigen_vects=3)

In [16]:
df.dtypes

vertex     int32
cluster    int32
dtype: object

In [17]:
# Check the edge cut score for the produced clustering
score = cugraph.analyzeClustering_edge_cut(G, 2, df['cluster'])
score

8.5

#### Define and print function, but adjust vertex ID so that they match the illustration

In [22]:
def print_cluster(df, id):
    
    # See which nodes are in cluster 0:
    part = []
    for i in range(len(df)):
        if (df['cluster'][i] == id):
            part.append(df['vertex'][i] + 1)
    print(part)

In [23]:
# See which nodes are in cluster 0:
print_cluster(df, 0)

[10, 27]


In [24]:
# See which nodes are in cluster 1:
print_cluster(df, 1)

[18, 22, 23, 25, 26, 28, 32]


In [25]:
# See which nodes are in cluster 2:
print_cluster(df, 2)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 24, 29, 30, 31, 33, 34]


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___